# Load data

You can save R objects to file and read them later to avoid running upstream steps over and over for your later exploratory data analysis!
- http://www.sthda.com/english/wiki/saving-data-into-r-data-format-rds-and-rdata

### 1. Initiate R 


Load rpy2 to exert R within ipython notebook. 

In [ ]:
# %load_ext rpy2.ipython

In [ ]:
# %%R 
library (GenomicFeatures)
library (tximport)

In [ ]:
library(tidyverse)

### 2. Load annotations

Salmon measure transcript level abundance and we need a `tx2gene` variable to match transcripts to genes to be able to get gene level abundance. Here, we are using R functionalities to extract this directly from a `.gtf` file annotation. This is a good exercise for similar tasks! 

- [`gencode.v34.basic.annotation.gtf.gz` FTP Link](https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_34/gencode.v34.basic.annotation.gtf.gz)

In [ ]:
# %%R 

GTF = '~/genomes/hg38/gencode.v34/gencode.v34.basic.annotation.gtf.gz'

txdb  = makeTxDbFromGFF(GTF,organism='Homo sapiens')

In [ ]:
# %%R 
# tx2gene objects 
k <- keys(txdb, keytype = "TXNAME")
tx2gene <- AnnotationDbi::select(txdb, k, "GENEID", "TXNAME")

### 3. Load salmon quants 

- [Link to example salmon results zip file](https://github.com/abearab/RNAseq-tutorials/blob/main/part-2-diff-exp/full-quants.zip?raw=true)

In [ ]:
# %%R
files <- list.files(path='./quants', pattern="quant.sf",full.names = TRUE, recursive=T)

files

In [ ]:
# %%R 
names(files) <- gsub("./quants/(\\S+)/quant.sf","\\1",files)

files

In [ ]:
# %%R 
txi <- tximport(files, type = "salmon", tx2gene = tx2gene, txOut=T)

In [ ]:
# %%R 
txi.gene <- summarizeToGene(txi, tx2gene, ignoreAfterBar= TRUE)

### 4. Define the sample sheet

In [ ]:
# %%R 
colnames(txi$abundance)

In [ ]:
# %%R 
hours = c(rep('120h', 4), rep('6h',4), rep('72h',4))
hours

In [ ]:
# %%R 
# meta 
treats  <- rep(c(rep('DMSO',2), rep('treated',2)),3)
reps    <- rep(c('rep1','rep2'),6)
hours   <- c(rep('120h',4),rep('6h',4),rep('72h',4))

colData <- data.frame(
    time=hours, 
    cond=treats, 
    sample_id=paste(hours, treats, sep='_'),
    row.names=colnames(txi$abundance))
colData

### 5. Make DESeq2 object 


> `DESeqDataSet` is a subclass of `RangedSummarizedExperiment`, used to store the input values, intermediate calculations and results of an analysis of differential expression. The `DESeqDataSet` class enforces non-negative integer values in the "counts" matrix stored as the first element in the assay list. ([link](https://rdrr.io/bioc/DESeq2/man/DESeqDataSet.html
))

In [ ]:
# %%R 
library(DESeq2)

In [ ]:
# %%R
dds0 <- DESeqDataSetFromTximport(txi.gene, colData, ~cond + time )

In [ ]:
# %%R 
class (dds0)

In [ ]:
# %%R
dim(dds0)

In [ ]:
# %%R 
saveRDS(dds0, 'dds0.rds')

### 6. Run `DEseq`

Filter genes with zero counts 

In [ ]:
# %%R 
dds <- DESeq(dds0)

In [ ]:
# %%R 
saveRDS(dds, 'dds.rds')

### 4. Get normalized counts 

In [ ]:
# %%R 
dds <- readRDS('dds.rds')

In [ ]:
# %%R 
ncu <- counts(dds, normalized=TRUE) 
head(ncu )

Save counts to a text file:

In [ ]:
# %%R 
write.table(ncu,'counts_DESeq2_norm.txt', sep="\t", quote=FALSE, col.names=row.names(colData))

### 6. Principal component analysis

In [ ]:
vsd <- varianceStabilizingTransformation(dds)

In [ ]:
# %%R 
z <- plotPCA(vsd,intgroup=c('time', 'cond'), returnData=TRUE)
percentVar <- round(100 * attr(z, "percentVar"))

Use [plotPCA](https://www.rdocumentation.org/packages/DESeq2/versions/1.12.3/topics/plotPCA) function and `ggplot2`. 

In [ ]:
# %%R 
library(ggplot2)
library(ggrepel)

In [ ]:
# %%R 
p <- ggplot(z, aes(PC1, PC2)) +
    geom_point(aes(size = 2), alpha = 4/10) +
    geom_text_repel(aes(label = row.names(colData)),size = 3.5) +
    xlab(paste0("PC1: ",percentVar[1],"% variance")) +
    ylab(paste0("PC2: ",percentVar[2],"% variance")) +
    guides (size = 'none') +
    ggtitle ("PCA plot") + 
    theme(legend.position="none")


In [ ]:
p

Above `p` variable is a `ggplot` object. Feel free to add more details or modify it to look better :)

Here is some useful links: 
- https://rafalab.github.io/dsbook/ggplot2.html

___
# 

In [ ]:
# %%R
sessionInfo()